In [ ]:
# 원본 영상을 입력받아, fps, 해상도를 변경하여 저장.

# 원본영상 불러오기

import os
import cv2

media_path = r'D:/development/2025\IUU\development\rtsp_simulator\media_source'

for file in os.listdir(media_path):
    if file.endswith('.mp4'):
        print(file)
        # 원본 영상 불러오기
        video_path = os.path.join(media_path, file)
        video = cv2.VideoCapture(video_path)
        # 원본 영상 정보 출력
        print(f"원본 영상 정보: {video.get(cv2.CAP_PROP_FRAME_COUNT)} 프레임, {video.get(cv2.CAP_PROP_FPS)} fps, {video.get(cv2.CAP_PROP_FRAME_WIDTH)}x{video.get(cv2.CAP_PROP_FRAME_HEIGHT)}")
        # 원본 영상 정보 출력   
        # 영상 변환 후 저장 15fps, 720x480
        # 영상 변환 후 저장 폴더 생성
        save_path = os.path.join(media_path, 'converted')
        os.makedirs(save_path, exist_ok=True)
        # 영상 변환 후 저장
        video.set(cv2.CAP_PROP_FPS, 15)
        video.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        video.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        video.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'mp4v'))
        
        # 저장 파일명 생성
        save_file_name = os.path.splitext(file)[0] + '_converted.mp4'
        save_file_path = os.path.join(save_path, save_file_name)
        # 저장
        out = cv2.VideoWriter(save_file_path, cv2.VideoWriter_fourcc(*'mp4v'), 15, (1280, 720))
        while True:
            ret, frame = video.read()
            if not ret:
                break
            out.write(frame)
        out.release()
        video.release()
        print(f"영상 변환 완료: {save_file_path}")

        

미디어1.mp4
원본 영상 정보: 331.0 프레임, 24.0 fps, 1280.0x720.0
영상 변환 완료: D:/development/2025\IUU\development\rtsp_simulator\media_source\converted\미디어1_converted.mp4
미디어2.mp4
원본 영상 정보: 1432.0 프레임, 24.003119958609144 fps, 1280.0x720.0
영상 변환 완료: D:/development/2025\IUU\development\rtsp_simulator\media_source\converted\미디어2_converted.mp4
미디어3.mp4
원본 영상 정보: 1718.0 프레임, 24.0 fps, 1280.0x720.0
영상 변환 완료: D:/development/2025\IUU\development\rtsp_simulator\media_source\converted\미디어3_converted.mp4


In [ ]:
import cv2
import os
import random
import numpy as np
from pathlib import Path

def create_video_from_images(image_dir, output_path, num_images=900, fps=15, resolution=(1280, 720)):
    """
    지정된 디렉토리에서 이미지를 랜덤하게 선택하여 MP4 영상을 생성합니다.
    
    Args:
        image_dir (str): 이미지가 있는 디렉토리 경로
        output_path (str): 출력할 MP4 파일 경로
        num_images (int): 선택할 이미지 개수 (기본값: 900)
        fps (int): 프레임 레이트 (기본값: 15)
        resolution (tuple): 영상 해상도 (width, height) (기본값: (1280, 720))
    """
    
    # 지원하는 이미지 확장자
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif']
    
    # 디렉토리에서 이미지 파일 찾기
    image_files = []
    for ext in image_extensions:
        image_files.extend(Path(image_dir).glob(f'*{ext}'))
        image_files.extend(Path(image_dir).glob(f'*{ext.upper()}'))
    
    print(f"총 {len(image_files)}개의 이미지 파일을 찾았습니다.")
    
    # 이미지가 충분하지 않은 경우 처리
    if len(image_files) < num_images:
        print(f"경고: 요청한 {num_images}개보다 적은 {len(image_files)}개의 이미지만 있습니다.")
        num_images = len(image_files)
    
    # 랜덤하게 이미지 선택
    selected_images = random.sample(image_files, num_images)
    print(f"{num_images}개의 이미지를 랜덤하게 선택했습니다.")
    
    # 비디오 코덱 설정 (MP4)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, resolution)
    
    print("영상 생성 중...")
    
    for i, image_path in enumerate(selected_images):
        try:
            # 이미지 읽기
            img = cv2.imread(str(image_path))
            
            if img is None:
                print(f"경고: {image_path} 이미지를 읽을 수 없습니다. 건너뜁니다.")
                continue
            
            # 이미지 크기를 지정된 해상도로 조정
            resized_img = cv2.resize(img, resolution)
            
            # 비디오에 프레임 추가
            out.write(resized_img)
            
            # 진행률 표시
            if (i + 1) % 50 == 0:
                print(f"진행률: {i + 1}/{num_images} ({(i + 1) / num_images * 100:.1f}%)")
                
        except Exception as e:
            print(f"에러 발생 - {image_path}: {e}")
            continue
    
    # 비디오 작성 완료
    out.release()
    print(f"영상이 성공적으로 생성되었습니다: {output_path}")
    print(f"영상 정보: {resolution[0]}x{resolution[1]}, {fps}fps, {num_images}프레임")

# 사용 예시
if __name__ == "__main__":
    # 입력 디렉토리 경로
    image_directory = r'D:\development\2025\IUU\development\rtsp_simulator\output_many_people'
    
    # 출력 파일 경로
    output_video = 'output_people_video.mp4'
    
    # 영상 생성
    create_video_from_images(
        image_dir=image_directory,
        output_path=output_video,
        num_images=900,
        fps=15,
        resolution=(1280, 720)
    )

총 4052개의 이미지 파일을 찾았습니다.
900개의 이미지를 랜덤하게 선택했습니다.
영상 생성 중...
진행률: 50/900 (5.6%)
진행률: 100/900 (11.1%)
진행률: 150/900 (16.7%)
진행률: 200/900 (22.2%)
진행률: 250/900 (27.8%)
진행률: 300/900 (33.3%)
진행률: 350/900 (38.9%)
진행률: 400/900 (44.4%)
진행률: 450/900 (50.0%)
진행률: 500/900 (55.6%)
진행률: 550/900 (61.1%)
진행률: 600/900 (66.7%)
진행률: 650/900 (72.2%)
진행률: 700/900 (77.8%)
진행률: 750/900 (83.3%)
진행률: 800/900 (88.9%)
진행률: 850/900 (94.4%)
진행률: 900/900 (100.0%)
영상이 성공적으로 생성되었습니다: output_people_video.mp4
영상 정보: 1280x720, 15fps, 900프레임


In [7]:
'''
fps (int): 프레임 레이트 (기본값: 15)
resolution (tuple): 영상 해상도 (width, height) (기본값: (1280, 720))
OPENCV를 사용하여 검정색 배경 영상 생성
'''

import cv2

def create_black_video(output_path, fps, resolution):
    # 영상 생성
    video = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, resolution)
    # 영상 저장
    for i in range(fps * 1):
        video.write(np.zeros((resolution[1], resolution[0], 3), dtype=np.uint8))
    video.release()
    print(f"검정색 배경 영상 생성 완료: {output_path}")
    
    
if __name__ == "__main__":
    output_path = "black_video.mp4"
    fps = 15
    resolution = (1280, 720)
    create_black_video(output_path, fps, resolution)









검정색 배경 영상 생성 완료: black_video.mp4


In [ ]:


import cv2
import os
import random
import numpy as np
from pathlib import Path


def create_video_sequence(image_dir, output_path,
                          num_images_per_batch=900,
                          batches=10,
                          black_duration_secs=1,
                          fps=30,
                          resolution=(1280, 720)):
    """
    1) 지정된 디렉토리에서 이미지를 랜덤하게 900개 선택하여 10회 반복
    2) 앞에 30fps 검은색 배경 영상을 2초(30프레임) 추가
    3) 총 9030프레임의 영상을 생성

    Args:
        image_dir (str): 이미지 디렉토리 경로
        output_path (str): 출력 비디오 파일 경로
        num_images_per_batch (int): 배치당 이미지 개수 (기본값: 900)
        batches (int): 배치 반복 횟수 (기본값: 10)
        black_duration_secs (int): 검은색 배경 지속 시간(초) (기본값: 1)
        fps (int): 출력 비디오 FPS (기본값: 30)
        resolution (tuple): 영상 해상도 (width, height)
    """
    # 이미지 파일 수집
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif']
    image_files = []
    p = Path(image_dir)
    for ext in image_extensions:
        image_files.extend(p.glob(f'*{ext}'))
        # image_files.extend(p.glob(f'*{ext.upper()}'))

    total_images = len(image_files)
    if total_images == 0:
        print(f"이미지 파일을 찾을 수 없습니다: {image_dir}")
        return

    print(f"총 {total_images}개의 이미지 파일을 찾았습니다.")

    # VideoWriter 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, resolution)

    # 검은색 배경 프레임 생성
    black_frame = np.zeros((resolution[1], resolution[0], 3), dtype=np.uint8)
    black_frames = black_duration_secs * fps
    print(f"검은색 배경: {black_duration_secs}초 ({black_frames}프레임) 추가")
    for _ in range(black_frames):
        out.write(black_frame)

    # 랜덤 이미지 배치 추가
    for batch_idx in range(1, batches+1):
        # 이미지 랜덤 선택
        if total_images < num_images_per_batch:
            selected = list(image_files)
            print(f"경고: 요청 {num_images_per_batch}개 > 실제 {total_images}개, 전체 사용")
        else:
            selected = random.sample(image_files, num_images_per_batch)
        print(f"배치 {batch_idx}/{batches}: {len(selected)}개 이미지 선택")

        for i, img_path in enumerate(selected):
            img = cv2.imread(str(img_path))
            if img is None:
                print(f"이미지 열기 실패, 건너뜀: {img_path}")
                continue
            # 해상도 맞추기
            frame = cv2.resize(img, resolution)
            out.write(frame)
            if (i+1) % 200 == 0:
                print(f"  진행: {i+1}/{num_images_per_batch} 프레임")

    out.release()
    print(f"영상 생성 완료: {output_path}")
    print(f"해상도: {resolution[0]}x{resolution[1]}, FPS: {fps}, 프레임: {black_frames + batches * num_images_per_batch}")


if __name__ == "__main__":
    # 실행 예시
    # image_directory = r'D:\development\2025\IUU\development\rtsp_simulator\test\output'
    # image_directory = r'D:\development\2025\IUU\development\rtsp_simulator\test\output_many_people'
    output_video = 'output_combined_peoples.mp4'
    create_video_sequence(
        image_dir=image_directory,
        output_path=output_video,
        num_images_per_batch=900,
        batches=10,
        black_duration_secs=2,
        fps=15,
        resolution=(1280, 720)
    )


총 4052개의 이미지 파일을 찾았습니다.
검은색 배경: 2초 (30프레임) 추가
배치 1/10: 900개 이미지 선택
  진행: 200/900 프레임
  진행: 400/900 프레임
  진행: 600/900 프레임
  진행: 800/900 프레임
배치 2/10: 900개 이미지 선택
  진행: 200/900 프레임
  진행: 400/900 프레임
  진행: 600/900 프레임
  진행: 800/900 프레임
배치 3/10: 900개 이미지 선택
  진행: 200/900 프레임
  진행: 400/900 프레임
  진행: 600/900 프레임
  진행: 800/900 프레임
배치 4/10: 900개 이미지 선택
  진행: 200/900 프레임
  진행: 400/900 프레임
  진행: 600/900 프레임
  진행: 800/900 프레임
배치 5/10: 900개 이미지 선택
  진행: 200/900 프레임
  진행: 400/900 프레임
  진행: 600/900 프레임
  진행: 800/900 프레임
배치 6/10: 900개 이미지 선택
  진행: 200/900 프레임
  진행: 400/900 프레임
  진행: 600/900 프레임
  진행: 800/900 프레임
배치 7/10: 900개 이미지 선택
  진행: 200/900 프레임
  진행: 400/900 프레임
  진행: 600/900 프레임
  진행: 800/900 프레임
배치 8/10: 900개 이미지 선택
  진행: 200/900 프레임
  진행: 400/900 프레임
  진행: 600/900 프레임
  진행: 800/900 프레임
배치 9/10: 900개 이미지 선택
  진행: 200/900 프레임
  진행: 400/900 프레임
  진행: 600/900 프레임
  진행: 800/900 프레임
배치 10/10: 900개 이미지 선택
  진행: 200/900 프레임
  진행: 400/900 프레임
  진행: 600/900 프레임
  진행: 800/900 프레임
영상 생성 완료: output_combin